In [247]:
import numpy as np
import pandas as pd
import statsmodels as sm
from sklearn.feature_selection import VarianceThreshold

In [248]:
met_df = pd.read_csv("../data/PL_MET_2024.csv", skiprows=[1,2]).convert_dtypes()
oc_df = pd.read_csv("../data/PL_OCEAN_2024.csv", skiprows=[1,2]).convert_dtypes()

In [249]:
met_df.columns = met_df.columns.str.lower().str.replace(" ", "_")
oc_df.columns = oc_df.columns.str.lower().str.replace(" ", "_")

In [250]:
met_df["time_(utc)"] = pd.to_datetime(met_df["time_(utc)"])
oc_df["time_(utc)"] = pd.to_datetime(oc_df["time_(utc)"])

/var/folders/rk/qwqd81bx00b1rl2fj8gy_vnc0000gn/T/ipykernel_67661/3744391951.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  met_df["time_(utc)"] = pd.to_datetime(met_df["time_(utc)"])
/var/folders/rk/qwqd81bx00b1rl2fj8gy_vnc0000gn/T/ipykernel_67661/3744391951.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  oc_df["time_(utc)"] = pd.to_datetime(oc_df["time_(utc)"])


In [251]:
merged_df = pd.merge(met_df, oc_df)

In [252]:
merged_df.describe(include='all').T

,count,mean,min,25%,50%,75%,max,std
time_(utc),44730,2024-04-03 13:11:11.959758080,2024-01-01 00:00:00,2024-02-16 20:19:30,2024-04-03 13:15:00,2024-05-20 07:10:30,2024-07-05 23:54:00,NaN
latitude,43700.0,38.032564,0.00002,38.0334,38.03341,38.03349,38.33342,0.18195
longitude,43700.0,-76.33564,-76.33781,-76.33749,-76.33733,-76.33731,0.00002,0.365183
air_temperature_qc,44730.0,1.238609,1.0,1.0,1.0,1.0,9.0,1.121977
air_temperature,44730.0,12.126716,-7.53,5.49,10.73,19.18,31.09,7.956304
air_pressure_qc,44730.0,1.238565,1.0,1.0,1.0,1.0,9.0,1.121947
air_pressure,44730.0,1016.714692,982.82,1012.0925,1016.59,1021.64,1057.09,8.083019
humidity_qc,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
humidity,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
wind_speed_qc,44730.0,1.104091,1.0,1.0,1.0,1.0,4.0,0.30677


In [253]:
qc_list = [col for col in merged_df.columns if "qc" in col]
merged_df = merged_df.drop(columns=qc_list)
merged_df.describe(include='all').T

,count,mean,min,25%,50%,75%,max,std
time_(utc),44730,2024-04-03 13:11:11.959758080,2024-01-01 00:00:00,2024-02-16 20:19:30,2024-04-03 13:15:00,2024-05-20 07:10:30,2024-07-05 23:54:00,NaN
latitude,43700.0,38.032564,0.00002,38.0334,38.03341,38.03349,38.33342,0.18195
longitude,43700.0,-76.33564,-76.33781,-76.33749,-76.33733,-76.33731,0.00002,0.365183
air_temperature,44730.0,12.126716,-7.53,5.49,10.73,19.18,31.09,7.956304
air_pressure,44730.0,1016.714692,982.82,1012.0925,1016.59,1021.64,1057.09,8.083019
humidity,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
wind_speed,43700.0,5.691698,0.12,3.57,5.44,7.38,31.1,2.944157
wind_direction,43700.0,184.431826,0.0,110.2,180.6,277.725,359.9,101.151468
temperature,44000.0,13.638423,3.08,6.94,10.34,19.94,27.95,7.434914
salinity,44000.0,10.809614,0.54,9.38,10.31,11.8925,15.78,1.824098


In [254]:
merged_df = merged_df.drop(columns=['humidity','chlorophyll', 'turbidity', 'oxygen', 'longitude', 'latitude'])

In [256]:
merged_df.describe(include='all').T

,count,mean,min,25%,50%,75%,max,std
time_(utc),44730,2024-04-03 13:11:11.959758080,2024-01-01 00:00:00,2024-02-16 20:19:30,2024-04-03 13:15:00,2024-05-20 07:10:30,2024-07-05 23:54:00,NaN
air_temperature,44730.0,12.126716,-7.53,5.49,10.73,19.18,31.09,7.956304
air_pressure,44730.0,1016.714692,982.82,1012.0925,1016.59,1021.64,1057.09,8.083019
wind_speed,43700.0,5.691698,0.12,3.57,5.44,7.38,31.1,2.944157
wind_direction,43700.0,184.431826,0.0,110.2,180.6,277.725,359.9,101.151468
temperature,44000.0,13.638423,3.08,6.94,10.34,19.94,27.95,7.434914
salinity,44000.0,10.809614,0.54,9.38,10.31,11.8925,15.78,1.824098
significant_wave_height,44566.0,0.2779,0.0,0.13,0.23,0.37,1.61,0.201196
wave_from_direction,44566.0,180.705349,0.0,113.2,161.7,263.1,359.9,90.869654
wave_period,44566.0,3.738267,0.0,2.17,2.63,3.45,20.0,3.701362
